# 緊急停止付き

In [2]:
# Set up
import hebi
from time import sleep
import math
from math import pi
import scipy.constants
import numpy as np

lookup = hebi.Lookup()
group = lookup.get_group_from_names(['Wheel', 'HEBI'], ['Right', 'IO_BOARD'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)

for entry in lookup.entrylist:
    print(entry)

group_fbk = hebi.GroupFeedback(group.size)

Group not found: Did you forget to set the module family and names above?
Family: Wheel Name: Right Mac Address: D8:80:39:A2:3A:66


AttributeError: 'NoneType' object has no attribute 'size'

In [7]:
def feedback_handler(group_fbk):

    # Container to the IO feedback
    io = group_fbk.io
    A1 = np.asarray(io.a.get_float(1))
    B1 = np.asarray(io.b.get_int(1))
    B2 = np.asarray(io.b.get_int(2))
    A1[0] = 0.0
    A1[1] = round(A1[1], 2) - 0.03

    # Sets the command lifetime to 100 milliseconds
    group.command_lifetime = 100
    
    if A1[1] <= 1.0:
        speed = -pi * 3 * (1 - A1[1])
    elif A1[1] >= 2.0:
        speed = pi * 3 * (A1[1] - 2)
    elif A1[1] > 3.0:
        speed = -pi * 3
    else:
        speed = 0
    
    if B1[1] == 1:
        speed = 0
    if B2[1] == 1:
        group.clear_feedback_handlers()
    """
    if deg >= 0:
        speed = pi * A1[1]
    elif deg < 0:
        speed = -pi * A1[1]
    """

    group_command = hebi.GroupCommand(group.size)

    group_command.velocity = speed #無限回転
    group.send_command(group_command)

SyntaxError: 'break' outside loop (<ipython-input-7-462e4ff9b6d3>, line 41)

In [6]:
# Sets the feedback frequency to 100 milliseconds
group.feedback_frequency = 100

group.add_feedback_handler(feedback_handler)

# ハンドル操作

In [1]:
# Set up
import hebi
from time import sleep
import math
from math import pi
import scipy.constants
import numpy as np

lookup = hebi.Lookup()
group = lookup.get_group_from_names(['HEBI', 'Wheel', 'Wheel'], ['IO_BOARD', 'Right', 'Left'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)

for entry in lookup.entrylist:
    print(entry)

group_fbk = hebi.GroupFeedback(group.size)

Family: HEBI Name: IO_BOARD Mac Address: D8:80:39:A1:A9:53
Family: Wheel Name: Right Mac Address: D8:80:39:5A:49:E1
Family: Wheel Name: Left Mac Address: 54:10:EC:31:9D:61


In [2]:
def feedback_handler(group_fbk):

    # Container to the IO feedback
    io = group_fbk.io
    A1 = np.asarray(io.a.get_float(1))
    B1 = np.asarray(io.b.get_int(1))
    B2 = np.asarray(io.b.get_int(2))
    A1[0] = round(A1[0], 2) - 0.03

    # Sets the command lifetime to 100 milliseconds
    group.command_lifetime = 100
    
    if A1[0] <= 0.5:
        speed = np.array([0, pi, -2*pi])
    elif A1[0] >= 3.0:
        speed = np.array([0, 2*pi, -pi])
    elif A1[0] < 3.0 and A1[0] > 0.5:
        speed = np.array([0, pi, -pi])
    
    if B1[0] == 1:
        speed = np.array([0, 0, 0])
    if B2[0] == 1:
        group.clear_feedback_handlers()
    """
    if deg >= 0:
        speed = pi * A1[1]
    elif deg < 0:
        speed = -pi * A1[1]
    """
    group_command = hebi.GroupCommand(group.size)

    group_command.velocity = speed #無限回転
    group.send_command(group_command)

In [3]:
# Sets the feedback frequency to 100 milliseconds
group.feedback_frequency = 100

group.add_feedback_handler(feedback_handler)

# 押したら後進

In [1]:
# Set up
import hebi
from time import sleep
import math
from math import pi
import scipy.constants
import numpy as np

lookup = hebi.Lookup()
#group = lookup.get_group_from_names(['X5-1'], ['X-00404'])
#group = lookup.get_group_from_names(['Wheel'], ['Right'])
group = lookup.get_group_from_names(['Wheel'], ['IO_BOARD', 'Right', 'Left'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)
    
#group_fbk = hebi.GroupFeedback(group.size)

In [48]:
def feedback_handler(group_fbk):

    """
    # Container to the IO feedback
    io = group_fbk.io
    B1 = np.asarray(io.b.get_int(1))
    B2 = np.asarray(io.b.get_int(2))
    """

    # Sets the command lifetime to 100 milliseconds
    group.command_lifetime = 100
        
    
    x = 0.3
    spring_constant = pi * x # rad/sec
    group_command = hebi.GroupCommand(group.size)
    #group_command.velocity = [0, spring_constant, -spring_constant]  # go
    group_command.velocity = spring_constant
    
    #if B1[0] == 1:
    #    group_command.velocity = [0, -spring_constant, spring_constant]  # go back

    group.send_command(group_command)
    
    
    #if B2[0] == 1:
    #    group.clear_feedback_handlers()

In [49]:
# Sets the feedback frequency to 100 milliseconds
group.feedback_frequency = 100

group.add_feedback_handler(feedback_handler)
sleep(10)
group.clear_feedback_handlers()

# 初期トルクを0にする

In [1]:
# Set up
import hebi
from time import sleep
import math
from math import pi
import scipy.constants
import numpy as np

lookup = hebi.Lookup()
#group = lookup.get_group_from_names(['X5-1'], ['X-00404'])
group = lookup.get_group_from_names(['Wheel'], ['Right'])
#group = lookup.get_group_from_names(['Wheel'], ['IO_BOARD', 'Right', 'Left'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)
    
group_fbk = hebi.GroupFeedback(group.size)

In [94]:
def feedback_handler(group_fbk):
    
    # Sets the command lifetime to 1000 milliseconds
    group.command_lifetime = 1000
        
    group_command = hebi.GroupCommand(group.size)
    group_command.reference_effort = 0.0000
    group_command.effort = 0.2

    #if group_fbk.effort[0] == group_fbk.effort_command[0]:
    #    group_command.effort = 0
    #    print('MATCH!!')
    #    group.clear_feedback_handlers()
        
    group.send_command(group_command)
    
    print('Fbk Efforts: {0:+.3f}, Fbk Command: {1}'.format(group_fbk.effort[0], group_fbk.effort_command[0]))

In [95]:
group.feedback_frequency = 1000
group.add_feedback_handler(feedback_handler)
sleep(5)
group.clear_feedback_handlers()

Fbk Efforts: +0.018, Fbk Command: 0.0
Fbk Efforts: +0.019, Fbk Command: 0.0
Fbk Efforts: +0.013, Fbk Command: 0.0
Fbk Efforts: +0.019, Fbk Command: 0.0
Fbk Efforts: +0.019, Fbk Command: 0.0
Fbk Efforts: +0.059, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.071, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.071, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.033, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.033, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.033, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.013, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.015, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.016, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.025, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.022, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.005, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.010, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.010, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.005, Fbk Command: 0.20000000298023224


Fbk Efforts: -0.002, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.002, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.009, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.012, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.012, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.011, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.009, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.016, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.017, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.017, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.009, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.014, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.017, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.019, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.022, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.018, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.004, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.002, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.006, Fbk Com

Fbk Efforts: -0.000, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.009, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.009, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.015, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.016, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.016, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.001, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.011, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.005, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.001, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.006, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.003, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.001, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.006, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.006, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.002, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.009, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.000, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.003, Fbk Com

Fbk Efforts: +0.025, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.025, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.031, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.017, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.013, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.013, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.011, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.008, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.014, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.014, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.015, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.010, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.027, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.027, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.026, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.033, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.028, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.019, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.012, Fbk Com

Fbk Efforts: +0.006, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.011, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.004, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.004, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.000, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.007, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.007, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.004, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.005, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.004, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.000, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.003, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.001, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.001, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.007, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.007, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.008, Fbk Command: 0.20000000298023224
Fbk Efforts: +0.016, Fbk Command: 0.20000000298023224
Fbk Efforts: -0.003, Fbk Com

In [98]:
group_command = hebi.GroupCommand(group.size)
group_command.reference_effort = 0.0000
group_command.effort = 0
group.send_command(group_command)

True

In [31]:
group_fbk.effort

array([nan], dtype=float32)

# キーボード入力

In [1]:
import hebi
from time import sleep
import math
from math import pi
import scipy.constants
import numpy as np
from getch import getch, pause

lookup = hebi.Lookup()
#group = lookup.get_group_from_names(['Wheel', 'HEBI'], ['Right', 'Left', 'IO_BOARD'])
group = lookup.get_group_from_names(['Wheel'], ['Right', 'Left'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)

for entry in lookup.entrylist:
    print(entry)

Family: Wheel Name: Right Mac Address: D8:80:39:A2:3A:66
Family: Wheel Name: Left Mac Address: 54:10:EC:31:35:41


In [3]:
def command(key):
    com = np.empty(2)
    y = 0.1
    if key == 119: # w 前進
        com[0] += 0.1
    elif key == 97: # a 左折
        com[1] += y
    elif key == 115: # s 停止
        com = np.zeros(2)
    elif key == 100: # d 右折
        com[1] = -y
    elif key == 120: # x 後退
        com[0] -= 0.1
    return com
    
def Stop(group_fbk):
    # Sets the command lifetime to 100 milliseconds
    group.command_lifetime = 100

    group_command = hebi.GroupCommand(group.size)
    
    #group_command.velocity = [0, 0, 0]  # stop
    com = command(115)
    x = com[0]
    y = com[1]
    group_command.velocity = [x]
    group.send_command(group_command)
    #print('x = {0}, y = {1}'.format(x, y))

In [4]:
def run():
    group_fbk = hebi.GroupFeedback(group.size)

    # Sets the feedback frequency to 100 milliseconds
    group.feedback_frequency = 100

    #print("=========================================\n     w     \n  a  s  d  \n     x     \n=========================================\n\nIf you want to quit, please press 'q' or 'Ctrl + c'.\n")
    
    while True: # Ctrl+c or q
        key = int(input())
        group.clear_feedback_handlers()
        
        if key == 119: # w 前進
            group.add_feedback_handler(Go)
            
        elif key == 97: # a 左折
            group.add_feedback_handler(Left)
            
        elif key == 115: # s 停止
            group.add_feedback_handler(Stop)
            
        elif key == 100: # d 右折
            group.add_feedback_handler(Right)
            
        elif key == 120: # x 後退
            group.add_feedback_handler(Back)
            
        elif key == 3 or key == 113: # Ctrl+c or q
            break
            
        else:
            message = 'You pressed: {0}, {1}'.format(chr(key), key)
            print(message)

In [ ]:
run()

     w     
  a  s  d  
     x     

If you want to quit, please press 'q' or 'Ctrl + c'.

115
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 

x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0.0
x = 0.0, y = 0